In [1]:
from datetime import timedelta

def daterange(start_date, end_date):
    current = start_date
    while current < end_date:
        yield current
        current = current + timedelta(days=1)


### Download

In [4]:

from src.training.extractors.vision_binance_extractor import VisionBinanceExtractor

from src.training.extractors.config import DataExtractionConfig

In [3]:
csv_filenames = []

binance_downloader = VisionBinanceExtractor(
    currency=DataExtractionConfig.CURRENCY,
    timestamp=DataExtractionConfig.TIMESTAMP,
)

for curr_date in daterange(DataExtractionConfig.START_DATE, DataExtractionConfig.END_DATE):
    print(curr_date)
    binance_downloader.download_file(report_date=curr_date)
    csv_filenames.append(
        binance_downloader.unzip_file(report_date=curr_date),
    )

2025-07-01 00:00:00
2025-07-02 00:00:00
2025-07-03 00:00:00
2025-07-04 00:00:00
2025-07-05 00:00:00
2025-07-06 00:00:00
2025-07-07 00:00:00
2025-07-08 00:00:00
2025-07-09 00:00:00


In [4]:
csv_filenames

['data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-01.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-02.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-03.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-04.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-05.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-06.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-07.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-08.csv',
 'data\\spot\\daily\\klines\\BTCUSDT\\3m\\BTCUSDT-3m-2025-07-09.csv']

### CreatePosition

#### Read data


In [2]:
from src.training.extractors.klines_preparer import KLinesStockHistoryExtractor
from src.training.extractors.config import DataExtractionConfig

In [3]:
from src.stock_history.klines import KLinesStockHistory


history_extractor = KLinesStockHistoryExtractor(
    currency=DataExtractionConfig.CURRENCY,
    timestamp=DataExtractionConfig.TIMESTAMP,
)

klines_window = KLinesStockHistory()

for curr_date in daterange(DataExtractionConfig.START_DATE_READ, DataExtractionConfig.END_DATE_READ):
    print(curr_date)
    klines_window = klines_window.concat(
        stock_history=history_extractor.extract_history(curr_date),
    )

2025-07-02 00:00:00
2025-07-03 00:00:00
2025-07-04 00:00:00
2025-07-05 00:00:00
2025-07-06 00:00:00
2025-07-07 00:00:00
2025-07-08 00:00:00


In [14]:
klines_window.get_window()['open_time']

0      1751414400000000
1      1751414580000000
2      1751414760000000
3      1751414940000000
4      1751415120000000
             ...       
475    1752018300000000
476    1752018480000000
477    1752018660000000
478    1752018840000000
479    1752019020000000
Name: open_time, Length: 3360, dtype: int64

In [16]:
import pandas as pd

pd.to_datetime(klines_window.get_window()['close_time'], unit='us')


0     2025-07-02 00:02:59.999999
1     2025-07-02 00:05:59.999999
2     2025-07-02 00:08:59.999999
3     2025-07-02 00:11:59.999999
4     2025-07-02 00:14:59.999999
                 ...            
475   2025-07-08 23:47:59.999999
476   2025-07-08 23:50:59.999999
477   2025-07-08 23:53:59.999999
478   2025-07-08 23:56:59.999999
479   2025-07-08 23:59:59.999999
Name: close_time, Length: 3360, dtype: datetime64[ns]